In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 12871900
paper_name = 'griffith_devine_2003' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, sep='\t')

In [13]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 101 x 2


In [14]:
original_data.head()

,0,1
0,ARD1,-1
1,NAT1,-1
2,SAP30,-1
3,SIN1 (SPT2),-1
4,SIN3,-1


In [15]:
original_data['gene'] = original_data[0].astype(str)

In [16]:
original_data['gene'] = original_data['gene'].apply(lambda x: x.split(' ')[0])

In [17]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [18]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [19]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                 0  1   gene    orf
index_input                        
22            FYV3 -1   FYV3   FYV3
26           KRE24 -1  KRE24  KRE24
32            SDF1 -1   SDF1   SDF1
50            TCI1 -1   TCI1   TCI1


In [20]:
original_data = original_data.loc[t,:]

In [21]:
original_data['data'] = pd.to_numeric(original_data[1], errors='coerce')

In [22]:
original_data.set_index('orf', inplace=True)

In [23]:
original_data = original_data[['data']].copy()

In [24]:
original_data = original_data.groupby(original_data.index).mean()

In [25]:
original_data.shape

(97, 1)

# Load & process tested strains

In [26]:
files = ['Res Gen diploid knock01.xlsx','Res Gen diploid knockouts02.xlsx']
sheets = ['Res Gen diploid knock01.txt','Res Gen diploid knockouts02.txt']

In [33]:
tested_list = []
for ixf, f in enumerate(files):
    tested = pd.read_excel('raw_data/' + f, sheet_name=sheets[ixf], skiprows=1)
#     print(tested.head())
    tested['orf'] = tested['ORF name'].astype(str)
    tested['orf'] = clean_orf(tested['orf'])
    
    typo_fixes = {'TAL004W':'YAL004W','YELOO1C':'YEL001C','KL187C':'YKL187C'}
    tested['orf'] = tested['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)
    tested['orf'] = translate_sc(tested['orf'], to='orf')
    # Make sure everything translated ok
    t = looks_like_orf(tested['orf'])
    print(tested.loc[~t,])
    tested = tested.loc[t,:]
    
    if 'plate' in tested.columns:
        c = 'plate'
    else:
        c = 'row'
    tested = tested.loc[(tested[c] >= 301) & (tested[c] <= 349)]
    tested_list.append(tested[['orf']])

                                        record no. ORF name   strain batch  \
index_input                                                                  
346                                          30827   YMR41W  HOM DIP  13_5   
3449         YPD +G418 +Riboflavin -glycerol media      NaN      NaN   NaN   
3603                             Q.C. Failures V.1      NaN      NaN   NaN   

             plate  row  column ResGen notes DB NOTES     orf  
index_input                                                    
346          304.0    F     6.0          NaN      NaN  YMR41W  
3449           NaN  NaN     NaN          NaN      NaN     NAN  
3603           NaN  NaN     NaN          NaN      NaN     NAN  
             record no. ORF name strain batch  row column ResGen notes  \
index_input                                                              
1042                NaN      NaN    NaN   NaN  NaN    NaN      end 349   
1109                NaN      NaN    NaN   NaN  NaN    NaN      end 

In [34]:
tested = pd.concat(tested_list, axis=0)

In [35]:
tested.head()

,orf
index_input,
0,YAL068C
1,YAL067C
2,YAL066W
3,YAL065C
4,YAL062W


In [36]:
tested_orfs = tested['orf'].unique()

In [37]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YML105C']

In [38]:
tested_orfs = list(tested_orfs) + missing

In [39]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [40]:
data = original_data.copy()

In [41]:
dataset_ids = [480]
datasets = datasets.reindex(index=dataset_ids)

In [42]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [43]:
data.head()

dataset_id,480
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [44]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [45]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,480
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [46]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [47]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [48]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        480          
data_type       value    valuez
gene_id orf                    
1869    YAL068C     0  0.068196
61      YAL067C     0  0.068196
60      YAL066W     0  0.068196
1727    YAL065C     0  0.068196
57      YAL062W     0  0.068196

# Print out

In [49]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [50]:
from IO.save_data_to_db3 import *

In [51]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 12871900...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.79s/it]

Updating the data_modified_on field...
